## Part 1

Idea: start at S. Perform a (depth-first?) search starting in all of the four initial directions that are possible. This will find you the path.

If there is a single furthest-away point regardless of direction chosen, then there must be an even number of parts in the loop, and we can find the distance by doing length(loop) / 2

Watch out! Indexing starts with the top-left value as `(0,0)` and increases downwards and rightwards. Give the row index `i` first, then column `j` (so effectively `(y, x)`).

In [12]:
with open('input.txt') as f:
    grid = [list(x) for x in f.read().splitlines()]

In [13]:
def get_start(grid: list[list[str]]):
    for i, row in enumerate(grid):
        if 'S' in row:
            return i, row.index('S')

In [14]:
pipe_connections = {
    'S': [(0, -1),  (0, 1), (1, 0), (-1, 0)],
    '|': [(1, 0), (-1, 0)],
    '-': [(0, 1), (0, -1)],
    'F': [(0, 1), (1, 0)],
    'L': [(0, 1), (-1, 0)],
    '7': [(0, -1), (1, 0)],
    'J': [(0, -1), (-1, 0)]
}

In [15]:
def get_adj_tiles(i: int, j: int):
    adj_tiles = []
    for i_offset, j_offset in pipe_connections[grid[i][j]]:
        if 0 <= i + i_offset < len(grid) and 0 <= j + j_offset < len(grid[0]): # check if in bounds
            adj_tile = grid[i + i_offset][j + j_offset]
            if adj_tile != '.': # i.e. it's a pipe
                if (-i_offset, -j_offset) in pipe_connections[adj_tile]:
                    adj_tiles.append((i + i_offset, j + j_offset))
    return adj_tiles

In [16]:
def dfs(start_tile: tuple[int, int]):
    stack = [(start_tile, [start_tile])]
    while stack:
        curr_tile, path = stack.pop()
        if len(path) != 1 and curr_tile == path[0]:
            return path[:-1]
        for adj_tile in get_adj_tiles(*curr_tile): 
            if len(path) < 2 or adj_tile != path[-2]: # don't revisit the tile we just came from
                stack.append((adj_tile, path + [adj_tile])) # if we're at a dead end, there won't be any adj_tiles to add to the stack, 
                                                            # so when it pops off the end of the stack again, it'll be backtracking to the last junction (since everything it visited got removed from the stack)
    return None

In [25]:
s_i, s_j = get_start(grid)
path = dfs((s_i, s_j))
len(path) / 2

6842.0

## Part 2

[Pick's theorem](https://en.wikipedia.org/wiki/Pick%27s_theorem) tells us that the area $A$ of a simple polygon with integer vertices is given by $A=i + \frac b 2 -1$ where $i = \text{number of interior points}$ and $b = \text {number of boundary points}$.

We can calculate $A$ easily using the [shoelace formula](https://en.wikipedia.org/wiki/Shoelace_formula). Then, we rearrange to get $i = A - \frac b 2 + 1$, where $b$ is just the length of our pipe

In [20]:
def get_area(polygon: list[tuple[int, int]]): # implementation of shoelace formula
    area = 0
    for i in range(len(polygon)):
        area += polygon[i][0] * polygon[(i + 1) % len(polygon)][1] - polygon[(i + 1) % len(polygon)][0] * polygon[i][1] # x_i * y_{i+1} - x_{i+1} * y_i
    return area / 2

In [21]:
def get_interior_points(polygon: list[tuple[int, int]]):
    return get_area(polygon) - len(polygon) / 2 + 1

In [24]:
get_interior_points(path)

393.0

In [18]:
def get_vertices(polygon: list[tuple[int, int]]):
    vertices = []
    for n, node in enumerate(path):
        prev = polygon[(n - 1) % len(polygon)]
        subs = polygon[(n + 1) % len(polygon)]
        s = (node[0] - prev[0], node[1] - prev[1])
        t = (subs[0] - node[0], subs[1] - node[1])
        assert subs == (prev[0] + s[0] + t[0], prev[1] + s[1] + t[1])
        if s[0] * t[0] + s[1] * t[1] == 0: # if s and t are perpendicular
            vertices.append(node)
    return vertices